This app is a chatbot that uses a deep learning model to respond to user inputs. The program is trained using the data in the ``intents.json'' file and gives appropriate answers to the user. In general, this program consists of several key sections:


data: https://www.kaggle.com/datasets/sheetaljade2019/intentsjson



1. **Data preprocessing**:
    - ``intents.json'' file contains several tags and text patterns (patterns) associated with each tag.
    - The data is parsed into text tokens (words) and converted to word roots using a stemmer.
    - The input data is displayed as a bag of words.

2. Modeling:
    - A deep learning model is defined using PyTorch, which includes an LSTM layer and a fully connected layer.
    - LSTM model is used to learn input sequences and better model relationships between words.
    - The output of the model is a probability distribution for each tag.


3. **model training**:
    - The model is trained using training data and optimization with Adam's method.
    - CrossEntropy cost function is used to evaluate the performance of the model.


4. **Prediction and response**:
    - User input becomes a bag of words.
    - The model predicts the most likely output as a label.
    - If the probability of prediction is higher than a certain limit, one of the responses related to that tag is selected from the ``intents.json'' file and displayed to the user.





In [16]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy as np
import random
import json
import pickle
import torch
import torch.nn as nn
import torch.optim as optim


In [17]:
# Download necessary NLTK data
nltk.download('punkt')

# Initialize the stemmer
stemmer = LancasterStemmer()

# Load the intents file
with open("intents.json") as file:
    data = json.load(file)

# Load or preprocess the data
try:
    with open("data.pickle", "rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []
        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [22]:
# Define the PyTorch model with LSTM
class ChatModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChatModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        h_0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
        c_0 = torch.zeros(1, x.size(0), hidden_size).to(x.device)
        output, (hn, cn) = self.lstm(x, (h_0, c_0))
        x = self.fc(output[:, -1, :])
        return x

# Hyperparameters
input_size = len(training[0])
hidden_size = 16  # Increased hidden size for better performance
output_size = len(output[0])
learning_rate = 0.001
num_epochs = 1000
batch_size = 8

# Prepare data
train_x = torch.Tensor(training).unsqueeze(1)  # Adding an extra dimension for LSTM
train_y = torch.Tensor(output)

# Initialize the model, loss function, and optimizer
model = ChatModel(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for i in range(0, len(train_x), batch_size):
        batch_x = train_x[i:i+batch_size]
        batch_y = train_y[i:i+batch_size]

        outputs = model(batch_x)
        loss = criterion(outputs, torch.max(batch_y, 1)[1])

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')

print("Model training complete.")

Epoch [100/1000], Loss: 0.2237
Epoch [200/1000], Loss: 0.0251
Epoch [300/1000], Loss: 0.0043
Epoch [400/1000], Loss: 0.0009
Epoch [500/1000], Loss: 0.0002
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
Model training complete.


In [25]:
# Save the model
torch.save(model.state_dict(), "model.pth")

# Function to convert user input to a bag of words
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    return np.array(bag)

# Chatting function
def chat():
    print("Start talking with the bot! (type 'quit' to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        bow = bag_of_words(inp, words)
        bow = torch.Tensor(bow).unsqueeze(0).unsqueeze(0)

        model.eval()
        with torch.no_grad():
            results = model(bow)
        results_index = torch.argmax(results)
        tag = labels[results_index.item()]

        if torch.max(results).item() > 0.8:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']
            print(random.choice(responses))
        else:
            print("I don't understand. Please ask something else.")

# Start chatting
chat()

Start talking with the bot! (type 'quit' to stop)
You: Hi
Good to see you again
You: who are you
I am Assist, a Deep-Learning chatbot.
You: whats deep learning
Deep learning differs from traditional machine learning in that it uses a hierarchical approach to learning, with multiple layers of artificial neurons, whereas traditional machine learning algorithms typically use simpler models.
You: quit
